In [ ]:
#STEP 1- Scraping WikiPedia and Creating Data Frame

# install and import libraries
#neighbourhood maps from open.toronto.ca


# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install html5lib 

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import folium
print('DONE!')

# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "4def3f65-2a65-4a4f-83c4-b2a4aed72d46"}
package = requests.get(url, params = params).json()
#print(package["result"])

In [146]:
# Getting the data, preprocessing, and cleaning the data

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url)
postal_code_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source.text, 'html')
soup1 = BeautifulSoup(postal_code_url,'lxml')
#print(soup.tbody.prettify())
#print("VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV")
#print(soup.tbody.prettify())

#using soup object, iterate the wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find("tbody")

#My_table = soup.find('tbody')
#df = pd.read_html(str(My_table))[0]
#df
#print(df)

#function for finding italics 
def has_italics_not_assigned(tag):
    return tag.has_attr('i') and not tag.has_attr('a')

def extract_postal(pcode):
    return pcode[0:3]

def extract_borough(borough):
    borough = borough[3:len(borough)]
    borough = borough.rpartition("(")[0]
    return borough

def extract_neighbourhood(nbhood):
    nbhood = nbhood.rpartition("(")[2]
    nbhood = nbhood.rpartition(")")[0]
    return nbhood

postals = []
boroughs = []
neighbourhoods =  []

for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
            td_string = td.text.strip()
            borough = td_string[3:len(td_string)]
            
            if (borough != "Not assigned"):
                section.append(extract_postal(td_string) + " " + borough)
                
                postals.append(extract_postal(td_string))
                boroughs.append(extract_borough(td_string))
                neighbourhoods.append(extract_neighbourhood(td_string))
            
            else:
                section.append("")
    #First row of data is the header
    """if (index == 0):
        columns = ['M1_', 'M2_', 'M3_', 'M4_', 'M5_', 'M6_', 'M7_', 'M8_', 'M9_']
    else:
        data.append(section)"""

#convert list into Pandas DataFrame
"""Toronto_df = pd.DataFrame(data = data, columns = columns)
Toronto_df"""

data = {"Postal Code": postals, "Borough": boroughs, "Neighbourhood": neighbourhoods}
Toronto_cleaned_df = pd.DataFrame(data)
Toronto_cleaned_df

#Toronto_df.shape
#Toronto_df = Toronto_df[Toronto_df['borough'] != '']
#Toronto_df.reset_index(inplace=True)
#Toronto_df.drop('index', axis=1, inplace=True)
#print('This table has {} rows and {} columns.'.format(Toronto_df.shape[0], Toronto_df.shape[1]))
#Toronto_df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [ ]:
# STEP 2: Getting the coordinates of each neighbourhood